In [1]:
import shutup
shutup.please()

import pandas as pd
import numpy as np
from datetime import datetime
import os, os.path, pickle, sys
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
census_population_dir = data_dir_base + "census/"
# Shannon_data_dir = data_dir_base + "Shannon_Data/"
# USDA_data_dir = data_dir_base + "/NASS_downloads/"
param_dir = data_dir_base + "parameters/"
Min_data_base = data_dir_base + "Min_Data/"
reOrganized_dir = data_dir_base + "reOrganized/"

In [3]:
county_id_name_fips = pd.read_csv(Min_data_base + "county_id_name_fips.csv")
county_id_name_fips.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

county_id_name_fips.sort_values(by=["state", "county"], inplace=True)

county_id_name_fips = rc.correct_Mins_FIPS(df=county_id_name_fips, col_="county")
county_id_name_fips.rename(columns={"county": "county_fips"}, inplace=True)

print (len(county_id_name_fips.state.unique()))
county_id_name_fips.head(2)

53


,county_fips,county_name,fips,state
0,02013,Aleutians East Borough,2013,AK
1,02016,Aleutians West Census Area,2016,AK


In [4]:
county_id_name_fips["state_fip"] = county_id_name_fips.county_fips.str.slice(0, 2)
county_id_name_fips = county_id_name_fips.drop(columns=["county_name", "county_fips", "fips"])
county_id_name_fips.drop_duplicates(inplace=True)
county_id_name_fips.reset_index(drop=True, inplace=True)
county_id_name_fips.head(2)

,state,state_fip
0,AK,02
1,AL,01


### read cattle inventory

In [5]:
shannon_annual = pd.read_csv(reOrganized_dir + "Shannon_Beef_Cows_fromCATINV.csv")
shannon_annual.reset_index(drop=True, inplace=True)
shannon_annual.head(2)

,state,1920,1921,1922,1923,1924,1925,1926,1927,1928,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AL,103000.0,106000.0,108000.0,111000.0,114000.0,104000.0,100000.0,96000.0,94000.0,...,680000.0,701000.0,681000.0,652000.0,673000.0,693000.0,714000.0,705000.0,696000.0,697000.0
1,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5300.0,5000.0,4300.0,4300.0,4000.0,4700.0,6100.0,6800.0,7700.0,7300.0


In [6]:
# form deltas: inventort(t+1) - inventory (t)
inv_deltas = shannon_annual[list(shannon_annual.columns)[2:]].values - \
               shannon_annual[list(shannon_annual.columns)[1:-1]].values

delta_columns = [(str(x) + "-" + str(x-1)) for x in np.arange(1921, 2022)]
# form deltas dataframe
shannon_annual_deltas = pd.DataFrame(data = inv_deltas, columns=delta_columns)
shannon_annual_deltas["state"] = shannon_annual["state"]

# re-order columns
shannon_annual_deltas = shannon_annual_deltas[["state"]+delta_columns]
shannon_annual_deltas.head(2)

,state,1921-1920,1922-1921,1923-1922,1924-1923,1925-1924,1926-1925,1927-1926,1928-1927,1929-1928,...,2012-2011,2013-2012,2014-2013,2015-2014,2016-2015,2017-2016,2018-2017,2019-2018,2020-2019,2021-2020
0,AL,3000.0,2000.0,3000.0,3000.0,-10000.0,-4000.0,-4000.0,-2000.0,-2000.0,...,21000.0,21000.0,-20000.0,-29000.0,21000.0,20000.0,21000.0,-9000.0,-9000.0,1000.0
1,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,200.0,-300.0,-700.0,0.0,-300.0,700.0,1400.0,700.0,900.0,-400.0


In [7]:
shannon_annual_deltas.head(2)

,state,1921-1920,1922-1921,1923-1922,1924-1923,1925-1924,1926-1925,1927-1926,1928-1927,1929-1928,...,2012-2011,2013-2012,2014-2013,2015-2014,2016-2015,2017-2016,2018-2017,2019-2018,2020-2019,2021-2020
0,AL,3000.0,2000.0,3000.0,3000.0,-10000.0,-4000.0,-4000.0,-2000.0,-2000.0,...,21000.0,21000.0,-20000.0,-29000.0,21000.0,20000.0,21000.0,-9000.0,-9000.0,1000.0
1,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,200.0,-300.0,-700.0,0.0,-300.0,700.0,1400.0,700.0,900.0,-400.0


### convert to tall format

In [8]:
years = list(shannon_annual_deltas.columns[1:])
num_years = len(years)

inventory_delta_tall = pd.DataFrame(data=None, index=range(num_years*len(shannon_annual_deltas.state.unique())), 
                                    columns=["state", "year", "inventory_delta"], 
                                    dtype=None, copy=False)

idx_ = 0 
for a_state in shannon_annual_deltas.state.unique():
    curr = shannon_annual_deltas[shannon_annual_deltas.state == a_state]
    inventory_delta_tall.loc[idx_: idx_ + num_years - 1 , "inventory_delta"] = curr[years].values[0]
    inventory_delta_tall.loc[idx_: idx_ + num_years - 1 , "state"] = a_state
    inventory_delta_tall.loc[idx_: idx_ + num_years - 1 , "year"] = years
    idx_ = idx_ + num_years
    
inventory_delta_tall.head(2)


,state,year,inventory_delta
0,AL,1921-1920,3000.0
1,AL,1922-1921,2000.0


In [9]:
inventory_delta_tall.tail(2)

,state,year,inventory_delta
5149,US,2020-2019,-352000.0
5150,US,2021-2020,-181100.0


In [10]:
shannon_annual_deltas = pd.merge(shannon_annual_deltas, county_id_name_fips, on = ["state"], how = "left")
shannon_annual_deltas.head(2)

,state,1921-1920,1922-1921,1923-1922,1924-1923,1925-1924,1926-1925,1927-1926,1928-1927,1929-1928,...,2013-2012,2014-2013,2015-2014,2016-2015,2017-2016,2018-2017,2019-2018,2020-2019,2021-2020,state_fip
0,AL,3000.0,2000.0,3000.0,3000.0,-10000.0,-4000.0,-4000.0,-2000.0,-2000.0,...,21000.0,-20000.0,-29000.0,21000.0,20000.0,21000.0,-9000.0,-9000.0,1000.0,01
1,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-300.0,-700.0,0.0,-300.0,700.0,1400.0,700.0,900.0,-400.0,02


In [11]:
inventory_delta_tall = pd.merge(inventory_delta_tall, county_id_name_fips, on = ["state"], how = "left")
inventory_delta_tall.head(2)

,state,year,inventory_delta,state_fip
0,AL,1921-1920,3000.0,01
1,AL,1922-1921,2000.0,01


In [12]:
from datetime import datetime
import pickle

filename = reOrganized_dir + "Shannon_Beef_Cows_fromCATINV_deltas.sav"

export_ = {"shannon_annual_inventory_deltas": shannon_annual_deltas, 
           "shannon_annual_inventory_deltas_tall": inventory_delta_tall, 
           "source_code" : "annual_state_level_inventory_deltas",
           "Author": "HN",
           "Date" : datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

pickle.dump(export_, open(filename, 'wb'))